# Transformer

[李沐讲论文]()

![](01.png)

1. 在计算下一个输出的时候，会把之前的输出也作为输入，这个就叫auto-regressive。也就是图中右半部分最下面output(shifted right)。比如输入英文，输出中文，那么会把前面输出的中文也作为解码器的输入

2. 就是因为有自回归，所以解码器在计算attention的时候，增加了mask，防止看到后面的内容

![](02.png)

1. Batch Norm 和 Layer Norm

以二维为例，每一行是一个样本，每个样本有d个feature。Batch Norm就是按照batch的维度去做normalization，也就是把不同batch里的相同feature位置的值作归一。也就是下图中的蓝线部分。而layer norm 是把一个样本中的不同feature的值去做归一，也就是下图中的黄线部分。

对于3维情况也是类似的（图中立方体），比如机器翻译，每次输入的是一个序列，每个位置是一个词向量。词向量的每个位置是一个feature。如果作batch norm，就是把相同feature位置所有batch的值去做归一（蓝线）。而layer norm是把同一个batch里面所有不同feature的值作归一（黄线）

![](03.png)


之所以用layer norm比较多，是因为输入的序列往往长度不一样。也就是不同的batch长度不一样。如果按照batch norm，那么空的地方要填入0，对计算均值和方差是有影响的。而layer norm不存在这个问题

![](04.png)

![](05.png)

1. 注意力机制就是计算某个向量$(query)$和其他一些已知向量$[(key_i,value_i)]$的相似度，然后作加权和。计算各个相似度用内积 $query \cdot key_i$，然后用这个相似度作权重（计算权重用softmax)，乘以$value_i$，再求和。

$ attention = \sum_{i=0}^n softmax((query \cdot key_i))  value_i $

2. 当有多个query的时候，可以用矩阵运算，也就是图中的公式

![](06.png)


3. 之所以要除$\sqrt d_k$是因为向量的维度比较大，点积之后的值可能很大或者很小。所softmax之后，就会非常接近1或者0,梯度比较小，不利于优化

![](07.png)
![](08.png)

1. 多头注意力机制和CNN里面的kernel是类似的。就是把Q、K、V通过一个矩阵作投影，这个投影矩阵的权重是可以学习的。作h次，就是h个不同的头

![](09.png)

1. 下面两个都是自注意力。key,value,query都是自己（比如英文翻译成中文，左边key,value,query就是一句话的各个词向量，都是一样的），所以叫做自注意力。左右区别在于右边有个mask，计算注意力的时候，只用当前词前面的词。这样在训练和测试的时候的行为是一致的

2. 绿框里里的key，value来自输入的输出，（也就是英文序列的词向量经过自注意力计算的结果），而query来自输出（也就是当前翻译产生的中文词向量）

3. 由于有attention机制，每个词向量已经包含了序列的信息。所以一个点的向量已经包含了序列信息，只需要用这个点的向量作query就可以了

![](10.png)

1. 因为之前的attention机制是没有时序信息的，打乱顺序对结果每影响。所以每个词向量需要加入position信息。也就是加入了一维数据作pos信息